<center><img src="https://storage.googleapis.com/unskript-website/assets/favicon.png" alt="unSkript.com" width="100" height="100">
<h1 id="unSkript-Runbooks">unSkript Runbooks<a class="jp-InternalAnchorLink" href="#unSkript-Runbooks" target="_self">&para;</a></h1>
<div class="alert alert-block alert-success">
<h3 id="Objective">Objective<a class="jp-InternalAnchorLink" href="#Objective" target="_self">&para;</a></h3>
<br><strong style="color: #000000;"><em>Fetch the Kubernetes system ConfigMap</em></strong></div>
</center><center>
<h2 id="Get-Kube-System-Config-Map"><u>Get Kube System Config Map</u><a class="jp-InternalAnchorLink" href="#Get-Kube-System-Config-Map" target="_self">&para;</a></h2>
</center>
<h1 id="Steps-Overview">Steps Overview<a class="jp-InternalAnchorLink" href="#Steps-Overview" target="_self">&para;</a></h1>
<p>1)&nbsp;<a href="#1" target="_self" rel="noopener">Get system config map</a><br>2)&nbsp;<a href="#2" target="_self" rel="noopener">Post slack message</a></p>

<h3 id="Get-List-of-Pods-in-ImagePullBackOff-State&para;"><a id="1" target="_self" rel="nofollow"></a>Convert namespace to String if empty<a class="jp-InternalAnchorLink" href="#Get-List-of-Pods-in-CrashLoopBackOff-State" target="_self" rel="noopener">&para;</a><a class="jp-InternalAnchorLink" href="#Get-List-of-Pods-in-ImagePullBackOff-State&para;" target="_self">&para;</a></h3>
<p>This custom action changes the type of namespace and config_map_name from None to String if no namespace is given</p>

In [3]:
if namespace == None:
    namespace = ''

<h3 id="List-all-IAM-Users"><a id="1" target="_self" rel="nofollow"></a>Get system config map</h3>
<p>This action gets the ConfigMap object for a given namespace or config map name. If neither is specified, namespace is considered to be "all".</p>
<blockquote>
<p>Action takes the following parameters (Optional) : <code>namespace, config_map_name</code><br>Action gives the following output (Optional) : <code>config_map_details</code></p>
</blockquote>

In [4]:
##
# Copyright (c) 2021 unSkript, Inc
# All rights reserved.
##
from typing import Optional, List, Tuple
from kubernetes import client
from pydantic import BaseModel, Field
from tabulate import tabulate
from unskript.legos.kubernetes.k8s_kubectl_command.k8s_kubectl_command import k8s_kubectl_command


from beartype import beartype
@beartype
def k8s_get_config_map_kube_system_printer(output):
    if output is None:
        return
    for x in output:
        for k,v in x.items():
            if k=='details':
                for config in v:
                    data_set_1 = []
                    data_set_1.append("Name:")
                    data_set_1.append(config.metadata.name)

                    data_set_2 = []
                    data_set_2.append("Namespace:")
                    data_set_2.append(config.metadata.namespace)

                    data_set_3 = []
                    data_set_3.append("Labels:")
                    data_set_3.append(config.metadata.labels)

                    data_set_4 = []
                    data_set_4.append("Annotations:")
                    data_set_4.append(config.metadata.annotations)

                    data_set_5 = []
                    data_set_5.append("Data:")
                    data_set_5.append(config.data)

                    tabular_config_map = []
                    tabular_config_map.append(data_set_1)
                    tabular_config_map.append(data_set_2)
                    tabular_config_map.append(data_set_3)
                    tabular_config_map.append(data_set_4)
                    tabular_config_map.append(data_set_5)

                    print(tabulate(tabular_config_map, tablefmt="github"))


@beartype
def k8s_get_config_map_kube_system(handle, config_map_name: str = '', namespace: str = None)->List:
    """k8s_get_config_map_kube_system get kube system config map

        :type handle: object
        :param handle: Object returned from the Task validate method

        :type config_map_name: str
        :param config_map_name: Kubernetes Config Map Name.

        :type namespace: str
        :param namespace: Kubernetes namespace.

        :rtype: List of system kube config maps for a given namespace
    """
    all_namespaces = [namespace]
    cmd = f"kubectl get ns  --no-headers -o custom-columns=':metadata.name'"
    if namespace is None or len(namespace)==0:
        kubernetes_namespaces = k8s_kubectl_command(handle=handle,kubectl_command=cmd )
        replaced_str = kubernetes_namespaces.replace("\n"," ")
        stripped_str = replaced_str.strip()
        all_namespaces = stripped_str.split(" ")
    result = []
    coreApiClient = client.CoreV1Api(api_client=handle)
    for n in all_namespaces:
        config_map_dict = {}
        res = coreApiClient.list_namespaced_config_map(
            namespace=n, pretty=True)
        if len(res.items) > 0:
            if config_map_name:
                config_maps = list(
                    filter(lambda x: (x.metadata.name == config_map_name), res.items))
            else:
                config_maps = res.items
            config_map_dict["namespace"] = n
            config_map_dict["details"] = config_maps
            result.append(config_map_dict)
    return result



task = Task(Workflow())
task.configure(inputParamsJson='''{
    "namespace": " namespace"
    }''')
task.configure(outputName="config_map_details")
task.configure(printOutput=True)
(err, hdl, args) = task.validate(vars=vars())
if err is None:
    task.execute(k8s_get_config_map_kube_system, lego_printer=k8s_get_config_map_kube_system_printer, hdl=hdl, args=args)

<h3 id="List-all-IAM-Users"><a id="1" target="_self" rel="nofollow"></a>Post a slack message</h3>
<p>This action posts a slack message of the config map retrieved in Step 1. This action will only run if the channel_name is specified in the parameters.</p>
<blockquote>
<p>Action takes the following parameters (Optional) : <code>channel_name, message</code></p>
</blockquote>

In [ ]:
##
# Copyright (c) 2021 unSkript, Inc
# All rights reserved.
##

import pprint

from pydantic import BaseModel, Field
from slack_sdk import WebClient
from slack_sdk.errors import SlackApiError

pp = pprint.PrettyPrinter(indent=2)

from beartype import beartype

from beartype import beartype
@beartype
def slack_post_message_printer(output):
    if output is not None:
        pprint.pprint(output)
    else:
        return


@beartype
@beartype
def slack_post_message(
        handle: WebClient,
        channel: str,
        message: str) -> str:

    try:
        response = handle.chat_postMessage(
            channel=channel,
            text=message)
        return f"Successfuly Sent Message on Channel: #{channel}"
    except SlackApiError as e:
        pp.pprint(
            f"Failed sending message to slack channel {channel}, Error: {e.response['error']}")
        if e.response['error'] == 'channel_not_found':
            raise Exception('Channel Not Found')
        elif e.response['error'] == 'duplicate_channel_not_found':
            raise Exception('Channel associated with the message_id not valid')
        elif e.response['error'] == 'not_in_channel':
            raise Exception('Cannot post message to channel user is not in')
        elif e.response['error'] == 'is_archived':
            raise Exception('Channel has been archived')
        elif e.response['error'] == 'msg_too_long':
            raise Exception('Message text is too long')
        elif e.response['error'] == 'no_text':
            raise Exception('Message text was not provided')
        elif e.response['error'] == 'restricted_action':
            raise Exception('Workspace preference prevents user from posting')
        elif e.response['error'] == 'restricted_action_read_only_channel':
            raise Exception('Cannot Post message, read-only channel')
        elif e.response['error'] == 'team_access_not_granted':
            raise Exception('The token used is not granted access to the workspace')
        elif e.response['error'] == 'not_authed':
            raise Exception('No Authtnecition token provided')
        elif e.response['error'] == 'invalid_auth':
            raise Exception('Some aspect of Authentication cannot be validated. Request denied')
        elif e.response['error'] == 'access_denied':
            raise Exception('Access to a resource specified in the request denied')
        elif e.response['error'] == 'account_inactive':
            raise Exception('Authentication token is for a deleted user')
        elif e.response['error'] == 'token_revoked':
            raise Exception('Authentication token for a deleted user has been revoked')
        elif e.response['error'] == 'no_permission':
            raise Exception('The workspace toekn used does not have necessary permission to send message')
        elif e.response['error'] == 'ratelimited':
            raise Exception('The request has been ratelimited. Retry sending message later')
        elif e.response['error'] == 'service_unavailable':
            raise Exception('The service is temporarily unavailable')
        elif e.response['error'] == 'fatal_error':
            raise Exception('The server encountered catostrophic error while sending message')
        elif e.response['error'] == 'internal_error':
            raise Exception('The server could not complete operation, likely due to transietn issue')
        elif e.response['error'] == 'request_timeout':
            raise Exception('Sending message error via POST: either message was missing or truncated')
        else:
            raise Exception(f'Failed Sending Message to slack channel {channel} Error: {e.response["error"]}')

    except Exception as e:
        print("\n\n")
        pp.pprint(
            f"Failed sending message to slack channel {channel}, Error: {e.__str__()}")
        return f"Unable to send message on {channel}"


task = Task(Workflow())
task.configure(inputParamsJson='''{
    "channel": "channel_name",
    "message": "f\\"Config map for namespace:{namespace}: {config_map_details}\\""
    }''')

task.configure(conditionsJson='''{
    "condition_enabled": true,
    "condition_cfg": "if len(channel_name)!=0",
    "condition_result": true
    }''')

task.configure(printOutput=True)
(err, hdl, args) = task.validate(vars=vars())
if err is None:
    task.execute(slack_post_message, lego_printer=slack_post_message_printer, hdl=hdl, args=args)

<h3 id="Conclusion">Conclusion</h3>
<p>In this Runbook, we were able to get the Kube ConfigMap and post a Slack message with the ConfigMap details. To view the full platform capabilities of unSkript please visit&nbsp;<a href="https://us.app.unskript.io" target="_blank" rel="noopener">us.app.unskript.io</a></p>